# Chapter 8
# Feature Selection For Machine Learning

The data features that you use to train your machine learning models have a huge influence on the performance you can achieve. Irrelevant or partially relevant features can negatively impact model performance. In this chapter you will discover automatic feature selection techniques that you can use to prepare your machine learning data in Python with scikit-learn. After completing this lesson you will know how to use:
1. Univariate Selection.
2. Recursive Feature Elimination.
3. Principle Component Analysis.
4. Feature Importance.

Let's get started.

## 8.1 Feature Selection

Feature selection is a <b>process where you automatically select those features in your data that contribute most to the prediction variable or output in which you are interested</b>. Having irrelevant features in your data can decrease the accuracy of many models, especially linear algorithms like linear and logistic regression. Three benefits of performing feature selection before modeling your data are:
- <b>Reduces Overfitting</b>: Less redundant data means less opportunity to make decisions based on noise.
- <b>Improves Accuracy</b>: Less misleading data means modeling accuracy improves.
- <b>Reduces Training Time</b>: Less data means that algorithms train faster.

## 8.2 Univariate Selection

Statistical tests can be used to select those features that have the strongest relationship with the output variable. The scikit-learn library provides the <b>SelectKBest</b> class that can be used with a suite of difierent statistical tests to select a specific number of features. The example below uses the chi-squared (chi2) statistical test for non-negative features to select 4 of the best features from the dataset.

In [20]:
# Feature Extraction with Univariate Statistical Tests (Chi-squared for classification)
from pandas import read_csv
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# load data
filename = '.\data\pima-indians-diabetes.data.csv'

names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8] # value to predict

# feature extraction: select 4 of the best features
test = SelectKBest(score_func=chi2, k=4)
fit = test.fit(X, Y)

# summarize scores
set_printoptions(precision=3)
print(fit.scores_)
features = fit.transform(X)

# summarize selected features
print(features[0:5,:])

[ 111.52  1411.887   17.605   53.108 2175.565  127.669    5.393  181.304]
[[148.    0.   33.6  50. ]
 [ 85.    0.   26.6  31. ]
 [183.    0.   23.3  32. ]
 [ 89.   94.   28.1  21. ]
 [137.  168.   43.1  33. ]]


You can see the scores for each attribute and the 4 attributes chosen (those with the highest scores): plas, test, mass and age. 

I got the names for the chosen attributes by manually mapping the index of the 4 highest scores to the index of the attribute names, as it's shown in the following:

- ['preg', <b>'plas'</b>, 'pres', 'skin', <b>'test'</b>, <b>'mass'</b>, 'pedi', <b>'age'</b>, 'class']
- [ 111.52 ~ <b>1411.887</b> ~ 17.605 ~ 53.108 ~ <b>2175.565</b> ~ <b>127.669</b> ~ 5.393 ~ <b>181.304</b>]

## 8.3 Recursive Feature Elimination

The Recursive Feature Elimination (or RFE) <b>works by recursively removing attributes and building a model on those attributes that remain. It uses the model accuracy to identify which  ttributes (and combination of attributes) contribute the most to predicting the target attribute</b>. You can learn more about the RFE class in the scikit-learn documentation. 

The example below uses RFE with the logistic regression algorithm to select the top 3 features. The choice of algorithm does not matter too much as long as it is skillful and consistent.

In [47]:
# Feature Extraction with RFE
from pandas import read_csv
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# load data
filename = '.\data\pima-indians-diabetes.data.csv'

names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]

# feature extraction
# To maintain the old behavior, you can specify the argument as follows:
model = LogisticRegression(solver='liblinear')

# To support the new behavior (recommended), you can specify the argument as follows:
# model = LogisticRegression(solver='lbfgs')

rfe = RFE(model, 3)
fit = rfe.fit(X, Y)

print("Num Features: %d" % fit.n_features_)
print("Selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % fit.ranking_)

Num Features: 3
Selected Features: [ True False False False False  True  True False]
Feature Ranking: [1 2 3 5 6 1 1 4]


You can see that RFE chose the top 3 features as preg, mass and pedi. These are <b>marked True in the support array and marked with a choice 1 in the ranking array</b>. Again, you can manually map the feature indexes to the indexes of attribute names.

- Num Features: 3
- Selected Features: [<b>True</b>, False, False, False, False, <b>True</b>, <b>True</b>, False]
- Feature Ranking: [<b>1</b>, 2, 3, 5, 6, <b>1</b>, <b>1</b>, 4]

## 8.4 Principal Component Analysis

Principal Component Analysis (or PCA) <b>uses linear algebra to transform the dataset into a compressed form</b>. Generally this is <a>called a data reduction technique</a>. A property of PCA is that you can choose the number of dimensions or principal components in the transformed result. In the example below, we use PCA and select 3 principal components.

In [48]:
# Feature Extraction with PCA
from pandas import read_csv
from sklearn.decomposition import PCA

# load data
filename = '.\data\pima-indians-diabetes.data.csv'

names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]

# feature extraction
pca = PCA(n_components=3)
fit = pca.fit(X)

# summarize components
print("Explained Variance: %s" % fit.explained_variance_ratio_)
print(fit.components_)

Explained Variance: [0.889 0.062 0.026]
[[-2.022e-03  9.781e-02  1.609e-02  6.076e-02  9.931e-01  1.401e-02
   5.372e-04 -3.565e-03]
 [-2.265e-02 -9.722e-01 -1.419e-01  5.786e-02  9.463e-02 -4.697e-02
  -8.168e-04 -1.402e-01]
 [-2.246e-02  1.434e-01 -9.225e-01 -3.070e-01  2.098e-02 -1.324e-01
  -6.400e-04 -1.255e-01]]


<b>Note</b>. You can see that the transformed dataset (3 principal components) bare little resemblance to the source data.

## 8.5 Feature Importance

Bagged decision trees like Random Forest and Extra Trees can be used to estimate the importance of features. In the example below we construct a <b>ExtraTreesClassifier</b> classifier for the dataset.

In [56]:
# Feature Importance with Extra Trees Classifier
from pandas import read_csv
from sklearn.ensemble import ExtraTreesClassifier

# load data
filename = '.\data\pima-indians-diabetes.data.csv'

names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]

# feature extraction

# To maintain the old behavior, you can specify the argument as follows:
# model = ExtraTreesClassifier(n_estimators=10)

# To support the new behavior (recommended), you can specify the argument as follows:
model = ExtraTreesClassifier(n_estimators=100)    
    
model.fit(X, Y)
print(model.feature_importances_)

[0.112 0.235 0.1   0.079 0.077 0.136 0.118 0.143]


You can see that we are given an importance score for each attribute where the larger the score, the more important the attribute. The scores suggest at the importance of plas, age and mass.

- ['preg', <b>'plas'</b>, 'pres', 'skin', 'test', <b>'mass'</b>, 'pedi', <b>'age'</b>, 'class']
- [0.112, <b>0.235</b>, 0.1, 0.079, 0.077, <b>0.136</b>, 0.118, <b>0.143</b>]